### Import necessary modules

In [ ]:
import unittest
import os
from datetime import datetime
from MainProgram import (
    Project,
    InvalidOrganizationException,
    InvalidCountryException,
    InvalidFocusAreaException,
    Validator
)

In [ ]:
import unittest
import os
from datetime import datetime
from A3 import (
    Project,
    InvalidOrganizationException,
    InvalidCountryException,
    InvalidFocusAreaException,
    Validator
)

class TestProject(unittest.TestCase):
    
    def setUp(self):
        """
        Set up test environment before each test case.
        Clears the project list and adds sample projects.
        """
        Project.projects.clear()  # Ensure no leftover data
        
        # Create sample projects for testing
        self.project1 = Project(
            project_name="Project Alpha",
            total_funding="100000",
            organization="Org A",
            focus_area="Health",
            country="United States",
            project_email="alpha@example.com",
            project_budget="$100,000.00",
            project_period="01/01/2024 ~ 31/12/2024"
        )
        self.project2 = Project(
            project_name="Project Beta",
            total_funding="200000",
            organization="Org B",
            focus_area="Education",
            country="United Kingdom",
            project_email="beta@example.com",
            project_budget="$200,000.00",
            project_period="01/02/2024 ~ 31/12/2024"
        )
        
        # Add sample projects to the project list
        Project.projects.append(self.project1)
        Project.projects.append(self.project2)
    
    def tearDown(self):
        """
        Clean up resources after each test case.
        """
        Project.projects.clear()  # Remove all projects after test
        # Remove temporary test files if they exist
        if os.path.exists("test_projects.txt"):
            os.remove("test_projects.txt")
        if os.path.exists("test_projects.json"):
            os.remove("test_projects.json")
    
    def test_write_projects_to_file(self):
        """
        Test if project data is correctly written to a text file.
        """
        filename = "test_projects.txt"
        Project.write_projects_to_file(filename)
        
        self.assertTrue(os.path.exists(filename))  # Ensure file exists
        with open(filename, 'r') as file:
            content = file.read()
            # Check if project details exist in file content
            self.assertIn("Project Name: Project Alpha", content)
            self.assertIn("Project Name: Project Beta", content)
            self.assertIn("Total Funding: 100000", content)
            self.assertIn("Total Funding: 200000", content)
    
    def test_load_projects_from_file(self):
        """
        Test loading project data from a text file.
        """
        filename = "test_projects.txt"
        Project.write_projects_to_file(filename)
        Project.projects.clear()  # Simulate a fresh load
        Project.load_projects_from_file(filename)
        
        self.assertEqual(len(Project.projects), 2)  # Ensure data is loaded correctly
        project_names = [project.getProjectName() for project in Project.projects]
        self.assertIn("Project Alpha", project_names)
        self.assertIn("Project Beta", project_names)
    
    def test_write_and_load_projects_to_json(self):
        """
        Test writing and loading project data in JSON format.
        """
        filename = "test_projects.json"
        Project.write_projects_to_json(filename)
        Project.projects.clear()
        Project.load_projects_from_json(filename)
        self.assertEqual(len(Project.projects), 2)
    
    def test_getters(self):
        """
        Test all getter methods of the Project class.
        """
        self.assertEqual(self.project1.getProjectName(), "Project Alpha")
        self.assertEqual(self.project1.getFunding(), "100000")
        self.assertEqual(self.project1.getOrganization(), "Org A")
        self.assertEqual(self.project1.getFocusArea(), "Health")
        self.assertEqual(self.project1.getCountry(), "United States")
    
    def test_setters(self):
        """
        Test all setter methods of the Project class.
        """
        self.project1.setFunding("150000")
        self.assertEqual(self.project1.getFunding(), "150000")
        self.project1.setOrganization("Org A Updated")
        self.assertEqual(self.project1.getOrganization(), "Org A Updated")
    
    def test_to_dict_and_from_dict(self):
        """
        Test serialization and deserialization of project objects.
        """
        project_dict = self.project1.to_dict()
        new_project = Project.from_dict(project_dict)
        self.assertEqual(new_project.getProjectName(), "Project Alpha")
    
    def test_search_by_name(self):
        """
        Test project search functionality by project name.
        """
        result = Project.search_by_name("Project Alpha")
        self.assertIsNotNone(result)
        self.assertEqual(result.getProjectName(), "Project Alpha")
        result = Project.search_by_name("Nonexistent Project")
        self.assertIsNone(result)
    
    def test_validate_email(self):
        """
        Test email validation function.
        """
        self.assertTrue(Validator.validate_email("test@example.com"))
        self.assertFalse(Validator.validate_email("invalid-email"))
    
    def test_parse_project_period(self):
        """
        Test parsing of project period into start and end dates.
        """
        start_date, end_date, duration = Validator.parse_project_period("01/01/2024 ~ 31/12/2024")
        self.assertEqual(start_date, datetime(2024, 1, 1))
        self.assertEqual(duration, 365)
    
    def test_invalid_organization_exception(self):
        """
        Test raising an exception for an invalid organization.
        """
        with self.assertRaises(InvalidOrganizationException):
            InvalidOrganizationException.validate_organization("Invalid Org")
    
    def test_load_projects_from_nonexistent_file(self):
        """
        Test loading from a non-existent file should not raise exceptions.
        """
        Project.projects.clear()
        Project.load_projects_from_file("nonexistent_file.txt")
        self.assertEqual(len(Project.projects), 0)
    
if __name__ == '__main__':
    unittest.main()